---------------------------------------------------------------------------

In [1]:
#| code-summary: Import Module

import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import xml.etree.ElementTree as ET
import warnings

warnings.filterwarnings("ignore")
pd.set_option('display.max_columns', None)
pd.set_option('display.precision', 10)

# Data Source

**Source : USGS(U.S. Geological Survey)[@hartsell2022climate]**

### Description:
The study aimed to assess changes in climate metrics from historical conditions across the Southeastern Utah Group of National Parks. The data include historical and projected future average temperatures for two emission scenarios and 12 global circulation models, with annual and seasonal averages. Data were obtained by sampling representative locations in each park and simulating daily variables using the SOILWAT2 model.

# Dataset Comparison (Clean vs Raw)

## Clean Data (Provided)

The clean data seems to have filtered out many columns and rows. In order to figure out what was filtered, I impolemented a few filters on the original dataset to see if I could get the same shape as the clean data.

1. Filter columns
2. Filter years 2021~2024

In [2]:
#| code-summary: Import Clean Data

clean_df = pd.read_csv('../data/nearterm_data_2020-2024.csv')

print("How does the clean dataset look like?")
clean_df.head()

How does the clean dataset look like?


,long,lat,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,DrySoilDays_Summer_whole,Evap_Summer,ExtremeShortTermDryStress_Summer_whole,FrostDays_Winter,NonDrySWA_Summer_whole,PPT_Winter,PPT_Summer,PPT_Annual,T_Winter,T_Summer,T_Annual,Tmax_Summer,Tmin_Winter,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole
0,-110.0472,37.60413,2021,NT,4.5,sc22,0,0,84,5,11,7,NaN,NaN,NaN,NaN,NaN,5.94,6.37,6.37,1.6303330,24.50402,24.50402,36.89,NaN,NaN,NaN,NaN,NaN
1,-110.0472,37.60413,2021,NT,4.5,sc22,0,0,84,5,11,7,0.0,3.2422296149,36.314,73.0,0.0929865127,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-12.77,0.1146518092,0.0787639891,0.0435142642,0.0512810069
2,-110.0472,37.60413,2021,NT,4.5,sc23,0,0,84,5,11,7,NaN,NaN,NaN,NaN,NaN,6.44,3.09,3.09,1.3890560,24.11043,24.11043,37.95,NaN,NaN,NaN,NaN,NaN
3,-110.0472,37.60413,2021,NT,4.5,sc23,0,0,84,5,11,7,0.0,2.4016114656,36.510,71.0,0.0001057892,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-18.96,0.1302210687,0.0964121637,0.0412322081,0.0922413330
4,-110.0472,37.60413,2021,NT,4.5,sc24,0,0,84,5,11,7,NaN,NaN,NaN,NaN,NaN,5.35,5.32,6.87,-0.3343889,25.54266,10.31321,37.74,NaN,NaN,NaN,NaN,NaN


In [3]:
#| code-summary: Import Raw Data and Filter

df = pd.read_csv('../data/NABR_ClimExposure', delimiter=' ')
clean_m = df[(df['year'] >= 2021) & (df['year'] <= 2024)]
clean_m = clean_m.loc[:, clean_m.columns.isin(list(clean_df.columns))]

print("Given Clean Dataset shape : ", clean_df.shape)
print("Filtered Original Dataset shape : ", clean_m.shape)

Given Clean Dataset shape :  (55802, 29)
Filtered Original Dataset shape :  (55802, 29)


## Conclusion
Since the clean data was adding two filters to the original dataset, I decided to clean the original dataset to extract as much information as possible.

# Raw Data Cleaning

In [4]:
#| code-summary: Import Original Dataset

# Convert to numeric, coercing errors to NaN
numeric_series = pd.to_numeric(df['RCP'], errors='coerce')

# Fill NaNs with original non-numeric values
df['RCP'] = numeric_series.fillna(df['RCP'])

df = df.drop_duplicates()

## Finding Patterns

First we will try to find a pattern in how to clean the dataset. What is the unique identity for each row?

By grouping vegetation, longitude, latitude, year, and scenario, we know that this isn't enough to get a unique row.<br>
Lets subset one combination to see what rows are in the given combination.

In [5]:
#| code-summary: Group Datasets

number_test = df.groupby(['veg','long','lat','year','scenario']).size().reset_index()
number_test.rename({0:'Size'},axis=1,inplace=True)
number_test[number_test['Size'] > 2].head()

,veg,long,lat,year,scenario,Size
1,Forest,-110.0348,37.59067,1981,sc1,5
3,Forest,-110.0348,37.59067,1983,sc1,5
5,Forest,-110.0348,37.59067,1985,sc1,5
7,Forest,-110.0348,37.59067,1987,sc1,5
9,Forest,-110.0348,37.59067,1989,sc1,5


## Subset Examination
By looking at the subset of a dataset below, we can see a few interesting points about this dataset

### Issue 1

**It seems like there are many duplicates but no rows are erased when erasing duplicates Why?**

 * For the first four rows, all the data seem to be duplicates. What's different?
    * The T_Annual and PPT_Annual seem to be different when all other features are the same.
    * T_Annual and PPT_Annual seem to be one of the season measurements.
    * A standard of unique rows must be defined for meaningful analysis.

### Solution 1

1. Remove T_Annual PPT_Annual columns
2. Remove duplicates
3. Find the sum of seasonal percipitation and re-define PPT_Annual
4. Find the average of seasonal temperatures and re-define T_Annual

In [6]:
#| code-summary: Data Subset of one specific identifier
df[(df['veg'] == 'Forest') & (df['year'] == 1981) & (df['long'] == -110.0348) & (df['lat'] == 37.59067) & (df['scenario'] == 'sc1')]

,Park,long,lat,veg,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,RL,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,PPT_Annual,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,T_Annual,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole
106077,NABR,-110.0348,37.59067,Forest,1981,Hist,historical,sc1,0,0,47,10,19,19,1792.537,72.9846,8.094261,2.230278,65.21617,634.6172,-1960.326,-9690.729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.37,10.16,9.69,12.45,9.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.393333,10.47446,23.1087,11.46374,3.393333,16.95,28.05,37.45,29.55,NaN,NaN,NaN,NaN,0.1318164,5.268698,4.032515,2.26957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106078,NABR,-110.0348,37.59067,Forest,1981,Hist,historical,sc1,0,0,47,10,19,19,1792.537,72.9846,8.094261,2.230278,65.21617,634.6172,-1960.326,-9690.729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.37,10.16,9.69,12.45,9.69,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.393333,10.47446,23.1087,11.46374,23.108700,16.95,28.05,37.45,29.55,NaN,NaN,NaN,NaN,0.1318164,5.268698,4.032515,2.26957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106079,NABR,-110.0348,37.59067,Forest,1981,Hist,historical,sc1,0,0,47,10,19,19,1792.537,72.9846,8.094261,2.230278,65.21617,634.6172,-1960.326,-9690.729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.37,10.16,9.69,12.45,2.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.393333,10.47446,23.1087,11.46374,3.393333,16.95,28.05,37.45,29.55,NaN,NaN,NaN,NaN,0.1318164,5.268698,4.032515,2.26957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106080,NABR,-110.0348,37.59067,Forest,1981,Hist,historical,sc1,0,0,47,10,19,19,1792.537,72.9846,8.094261,2.230278,65.21617,634.6172,-1960.326,-9690.729,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.37,10.16,9.69,12.45,2.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.393333,10.47446,23.1087,11.46374,23.108700,16.95,28.05,37.45,29.55,NaN,NaN,NaN,NaN,0.1318164,5.268698,4.032515,2.26957,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
106081,NABR,-110.0348,37.59067,Forest,1981,Hist,historical,sc1,0,0,47,1

In [21]:
#| code-summary: Data Cleaning 1

# Columns to exclude
cols_to_exclude = ['PPT_Annual', 'T_Annual']

# Create a boolean mask for columns to keep
mask = df.columns.isin(cols_to_exclude)

# Use loc to select all rows and only the columns not in the mask
new_df = df.loc[:, ~mask]

test = new_df.drop_duplicates()

number = test.groupby(['veg','year','long','lat','scenario']).size().reset_index()
number.rename({0:'Size'},axis=1,inplace=True)

# number[number['Size'] > 2]

test['PPT_Annual'] = test['PPT_Winter'] + test['PPT_Spring'] + test['PPT_Summer'] + test['PPT_Fall']
test['T_Annual'] = (test['T_Winter'] + test['T_Spring'] + test['T_Summer'] + test['T_Fall'])/4

### Issue 2

**Missing Column values as if extracted by two different sources at the same site**

 * Although the so called "identifiers" that I defined were the same, a the value of a group of columns were stored in row A and another group of columns were stored in row B.
   * Row A and Row B have identical identifiers.
   * When a column has a value in row A, row B has a null value in the same column
   * The RL column (Depth of Restriction Layer) is slightly different for the two identifiers (Same to fifth decimal point)
   * Since the RL was similar, I was willing to sacrifice the fifth decimal point precision for ease of analysis.

### Solution 2

1. Remove RL column but store if with the identifiers
2. Group by the identifiers and fine the average RL.
3. With the original dataset with RL removed, Merge the two rows together vertically so that the null values and data points for identical identifiers merge together.
4. join the average RL and original dataset with identifiers as the joinint key

In [26]:
#| code-summary: Before cleaning
test.head(10)

,Park,long,lat,veg,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,RL,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual
0,NABR,-110.0472,37.60413,Shrubland,1980,Hist,historical,sc1,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,54.57202000,1949.283,-8753.784,4834.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,13.79,8.71,2.69,6.37,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.96483520,8.767935,23.15924,11.962090,14.15,28.75,37.05,31.15,NaN,NaN,NaN,NaN,0.2370806,5.296833,1.067496,1.9667860,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,31.56,11.21352505
4,NABR,-110.0472,37.60413,Shrubland,1980,Hist,historical,sc1,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,54.57202148,1949.283,-8753.784,4834.13,-0.6636760860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7140658366,6.3995308949,1.5598074021,3.3632779979,NaN,24.34,36.16,29.52,NaN,24.34,36.16,29.52,75.0,34.0,0.0,26.0,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,7.7811633032,31.1394527955,48.0177480655,21.9156825756,NaN,NaN,NaN,NaN,36.5000000000,36.5000000000,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,-12.45,-7.35,5.55,-10.25,NaN,NaN,NaN,NaN,0.1134468701,0.0968307001,0.0418759016,0.0522975530,0.1134468701,0.0968307001,0.0418759016,0.0522975530,91.0,77.0,5.0,47.0,91.0,77.0,5.0,47.0,NaN,NaN
5,NABR,-110.0472,37.60413,Shrubland,1981,Hist,historical,sc1,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,54.57202000,1949.283,-8753.784,4834.13,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2.25,9.81,9.39,11.75,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,3.33444400,10.548370,23.27065,11.581320,17.05,28.15,37.55,29.75,NaN,NaN,NaN,NaN,0.2930753,3.506108,3.916328,2.7875470,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,33.20,12.18369600
6,NABR,-110.0472,37.60413,Shrubland,1981,Hist,historical,sc1,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,54.57202148,1949.283,-8753.784,4834.13,0.3478010620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1815202084,5.9723378265,5.0428776741,4.6374034668,13.92,26.53,36.08,NaN,13.92,26.53,36.08,NaN,79.0,26.0,0.0,13.0,0.3461917

In [29]:
#| code-summary: Data Cleaning
a = test.drop('RL', axis=1)
merged_df = a.groupby(list(a.columns[0:21]), as_index=False).agg(lambda x: next(iter(x.dropna()), np.nan))

In [7]:
#| code-summary: Data Cleaning
rl = test.groupby(['veg','year','long','lat','scenario'])['RL'].mean().reset_index()
merged_df = pd.merge(merged_df, rl, on=['veg', 'year', 'long', 'lat', 'scenario'], how='inner')

In [44]:
#| code-summary: Sanity Check
print("Original dataset rows with RL removed : ", merged_df.shape[0])
print("RL grouped and averaged by identifiers : ", rl.shape[0])

Original dataset rows with RL removed :  361487
RL grouped and averaged by identifiers :  361487


In [45]:
#| code-summary: Cleaned Data
merged_df.head()

,Park,long,lat,veg,year,TimePeriod,RCP,scenario,treecanopy,Ann_Herb,Bare,Herb,Litter,Shrub,El,Sa,Cl,RF,Slope,E,S,T_P_Corr,DrySoilDays_Winter_top50,DrySoilDays_Spring_top50,DrySoilDays_Summer_top50,DrySoilDays_Fall_top50,DrySoilDays_Winter_whole,DrySoilDays_Spring_whole,DrySoilDays_Summer_whole,DrySoilDays_Fall_whole,Evap_Winter,Evap_Spring,Evap_Summer,Evap_Fall,ExtremeShortTermDryStress_Winter_top50,ExtremeShortTermDryStress_Spring_top50,ExtremeShortTermDryStress_Summer_top50,ExtremeShortTermDryStress_Fall_top50,ExtremeShortTermDryStress_Winter_whole,ExtremeShortTermDryStress_Spring_whole,ExtremeShortTermDryStress_Summer_whole,ExtremeShortTermDryStress_Fall_whole,FrostDays_Winter,FrostDays_Spring,FrostDays_Summer,FrostDays_Fall,NonDrySWA_Winter_top50,NonDrySWA_Spring_top50,NonDrySWA_Summer_top50,NonDrySWA_Fall_top50,NonDrySWA_Winter_whole,NonDrySWA_Spring_whole,NonDrySWA_Summer_whole,NonDrySWA_Fall_whole,PET_Winter,PET_Spring,PET_Summer,PET_Fall,PPT_Winter,PPT_Spring,PPT_Summer,PPT_Fall,SemiDryDuration_Annual_top50,SemiDryDuration_Annual_whole,SWA_Winter_top50,SWA_Spring_top50,SWA_Summer_top50,SWA_Fall_top50,SWA_Winter_whole,SWA_Spring_whole,SWA_Summer_whole,SWA_Fall_whole,T_Winter,T_Spring,T_Summer,T_Fall,Tmax_Winter,Tmax_Spring,Tmax_Summer,Tmax_Fall,Tmin_Winter,Tmin_Spring,Tmin_Summer,Tmin_Fall,Transp_Winter,Transp_Spring,Transp_Summer,Transp_Fall,VWC_Winter_top50,VWC_Spring_top50,VWC_Summer_top50,VWC_Fall_top50,VWC_Winter_whole,VWC_Spring_whole,VWC_Summer_whole,VWC_Fall_whole,WetSoilDays_Winter_top50,WetSoilDays_Spring_top50,WetSoilDays_Summer_top50,WetSoilDays_Fall_top50,WetSoilDays_Winter_whole,WetSoilDays_Spring_whole,WetSoilDays_Summer_whole,WetSoilDays_Fall_whole,PPT_Annual,T_Annual,RL
0,NABR,-110.0472,37.60413,Shrubland,1980,Hist,historical,sc1,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,-0.6636760860,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.7140658366,6.3995308949,1.5598074021,3.3632779979,NaN,24.34,36.16,29.52,NaN,24.34,36.16,29.52,75.0,34.0,0.0,26.0,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,7.7811633032,31.1394527955,48.0177480655,21.9156825756,13.79,8.71,2.69,6.37,36.5000000000,36.5000000000,3.4668806371,2.6546632530,0.0321140671,0.4880867481,3.4668806371,2.6546632530,0.0321140671,0.4880867481,0.96483520,8.767935,23.15924,11.962090,14.15,28.75,37.05,31.15,-12.45,-7.35,5.55,-10.25,0.2370806,5.296833,1.067496,1.9667860,0.1134468701,0.0968307001,0.0418759016,0.0522975530,0.1134468701,0.0968307001,0.0418759016,0.0522975530,91.0,77.0,5.0,47.0,91.0,77.0,5.0,47.0,31.56,11.21352505,54.57202074
1,NABR,-110.0472,37.60413,Shrubland,1981,Hist,historical,sc1,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,0.3478010620,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.1815202084,5.9723378265,5.0428776741,4.6374034668,13.92,26.53,36.08,NaN,13.92,26.53,36.08,NaN,79.0,26.0,0.0,13.0,0.3461917264,0.8982752558,0.0336629893,2.5013360811,0.3461917264,0.8982752558,0.0336629893,2.5013360811,8.1229049607,32.3882557036,48.1772426406,21.7575735702,2.25,9.81,9.39,11.75,13.2500000000,13.2500000000,0.3461917264,0.8982752558,0.0336629893,2.5013360811,0.3461917264,0.8982752558,0.0336629893,2.5013360811,3.33444400,10.548370,23.27065,11.581320,17.05,28.15,37.55,29.75,-9.35,-5.55,1.25,-7.25,0.2930753,3.506108,3.916328,2.7875470,0.0493818430,0.0607271763,0.0426386771,0.0936706801,0.0493818430,0.0607271763,0.0426386771,0.0936706801,48.0,60.0,13.0,85.0,48.0,60.0,13.0,85.0,33.20,12.18369600,54.57202074
2,NABR,-110.0472,37.60413,Shrubland,1982,Hist,historical,sc1,0,0,84,5,11,7,1764.955,77.03307,6.082058,2.285707,1949.283,-8753.784,4834.13,0.3260300992,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.2589947135,4.7173273934,4.5276363327,4.2477717540,NaN,26.19,34.99,22.06,NaN,26.19,34.99,22.06,83.0,21.0,0.0,30.0,3.2599844936,1.5994982052,0.1993822366,1.2432253150,3.2599844936,1.5994982052,0.1993822366,1.2432253150,7.3379526955,31.4894498184,47.1800768757,21.0684231651,4.

In [35]:
#| code-summary: Output to CSV
# merged_df.to_csv('../data/cleaned_df.csv', index=False)